In [ ]:
# !pip install narwhals pandas polars pyarrow


In [2]:
import pandas as pd
import numpy as np
import narwhals as nw
import polars as pl

In [3]:
# Create synthetic dataset using pandas (as a native DataFrame)
num_rows = 200_000_000  # 200M rows for demonstration
rng = np.random.default_rng(seed=42)
pdf_sales = pd.DataFrame({
    "id": rng.integers(0, 20_000, size=num_rows),        # many repeat IDs to allow groupby
    "value": rng.normal(loc=100.0, scale=50.0, size=num_rows).round(2),  # some numeric value
})
# Create a region mapping: each id gets a region label (e.g., "North","South","East","West")
unique_ids = pdf_sales["id"].unique()
regions = ["North", "South", "East", "West"]
id_to_region = {id_val: rng.choice(regions) for id_val in unique_ids}
pdf_regions = pd.DataFrame({
    "id": list(id_to_region.keys()),
    "region": [id_to_region[i] for i in id_to_region.keys()]
})
# Quick peek at data shape
print(pdf_sales.shape, pdf_regions.shape)
print(pdf_sales.head(3), "\n", pdf_regions.head(3))
print(pdf_sales.info())

(200000000, 2) (20000, 2)
      id   value
0   1785   54.31
1  15479  123.47
2  13091  -15.55 
       id region
0   1785  North
1  15479   West
2  13091  North
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000000 entries, 0 to 199999999
Data columns (total 2 columns):
 #   Column  Dtype  
---  ------  -----  
 0   id      int64  
 1   value   float64
dtypes: float64(1), int64(1)
memory usage: 3.0 GB
None


In [4]:
print(pdf_regions.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20000 non-null  int64 
 1   region  20000 non-null  object
dtypes: int64(1), object(1)
memory usage: 312.6+ KB
None


In [5]:
# Convert to Polars DataFrames for demonstration
pl_sales = pl.DataFrame(pdf_sales)
pl_regions = pl.DataFrame(pdf_regions)

In [6]:
# Wrap the pandas dataframes with Narwhals
df_sales_nw = nw.from_native(pdf_sales)
df_regions_nw = nw.from_native(pdf_regions)

In [7]:
  # For a comprehensive overview similar to pandas .info()
def polars_info(df):
    print(f"Shape: {df.shape}")
    print(f"Memory usage: {df.estimated_size() / (1024**2):.2f} MB")
    print("\nColumn info:")
    for name, dtype in df.schema.items():
        print(f"  {name}: {dtype}")

In [8]:
polars_info(df_sales_nw)
polars_info(df_regions_nw)

Shape: (200000000, 2)
Memory usage: 3051.76 MB

Column info:
  id: Int64
  value: Float64
Shape: (20000, 2)
Memory usage: 1.63 MB

Column info:
  id: Int64
  region: String


In [9]:
import time
start_time = time.time()
# 1. Join sales with region labels on 'id'
df_joined = df_sales_nw.join(df_regions_nw, on="id", how="inner")
print("After join, columns:", df_joined.columns)
print("Sample row (native):", nw.to_native(df_joined).iloc[0].to_dict())
# 2. Add a derived column: value normalized by overall mean
overall_mean = pdf_sales["value"].mean()  # compute using pandas for reference
df_joined = df_joined.with_columns(
    (nw.col("value") / overall_mean).alias("value_norm")
)
# 3. Filter rows: keep only transactions with positive value
df_joined = df_joined.filter(nw.col("value") > 0)
print("Post-filter shape (native):", nw.to_native(df_joined).shape)
# 4. Group by region and aggregate total and average value
df_summary = df_joined.group_by("region").agg(
    nw.col("value").sum().alias("total_value"),
    nw.col("value").mean().alias("avg_value"),
    nw.len().alias("transaction_count")
)
# Convert result to native pandas for display
summary_native = nw.to_native(df_summary)
print("Summary by region (pandas):\n", summary_native)
end_time = time.time()
print(f"Execution time: {end_time - start_time:.4f} seconds")

After join, columns: ['id', 'value', 'region']
Sample row (native): {'id': 1785, 'value': 54.31, 'region': 'North'}
Post-filter shape (native): (195448682, 4)
Summary by region (pandas):
   region   total_value   avg_value  transaction_count
0  North  5.038577e+09  102.766448           49029396
1   West  5.101366e+09  102.751330           49647685
2   East  4.878919e+09  102.760417           47478583
3  South  5.065127e+09  102.755460           49293018
Execution time: 39.0191 seconds


In [15]:
print(type(df_sales_nw))

<class 'narwhals.dataframe.DataFrame'>


In [10]:
start_time = time.time()
# Verification using direct pandas (should match summary_native)
check = pdf_sales.merge(pdf_regions, on="id").query("value > 0").groupby("region")["value"].agg(['sum','mean','count'])
print(check.reset_index())
end_time = time.time()
print(f"Execution time: {end_time - start_time:.4f} seconds")


  region           sum        mean     count
0   East  4.878919e+09  102.760417  47478583
1  North  5.038577e+09  102.766448  49029396
2  South  5.065127e+09  102.755460  49293018
3   West  5.101366e+09  102.751330  49647685
Execution time: 34.1956 seconds


In [16]:
print(type(check))

<class 'pandas.core.frame.DataFrame'>


In [11]:
start_time = time.time()
# Using Polars directly (no Narwhals) for comparison:
pl_summary = (pl_sales.join(pl_regions, on="id", how="inner")
                        .filter(pl.col("value") > 0)
                        .with_columns((pl.col("value") / pl.col("value").mean()).alias("value_norm"))
                        .group_by("region")
                        .agg([
                            pl.col("value").sum().alias("total_value"),
                            pl.col("value").mean().alias("avg_value"),
                            pl.col("value").len().alias("transaction_count")
                        ])
             )
print("Summary by region (Polars):\n", pl_summary)
end_time = time.time()
print(f"Execution time: {end_time - start_time:.4f} seconds")

Summary by region (Polars):
 shape: (4, 4)
┌────────┬─────────────┬────────────┬───────────────────┐
│ region ┆ total_value ┆ avg_value  ┆ transaction_count │
│ ---    ┆ ---         ┆ ---        ┆ ---               │
│ str    ┆ f64         ┆ f64        ┆ u32               │
╞════════╪═════════════╪════════════╪═══════════════════╡
│ South  ┆ 5.0651e9    ┆ 102.75546  ┆ 49293018          │
│ North  ┆ 5.0386e9    ┆ 102.766448 ┆ 49029396          │
│ West   ┆ 5.1014e9    ┆ 102.75133  ┆ 49647685          │
│ East   ┆ 4.8789e9    ┆ 102.760417 ┆ 47478583          │
└────────┴─────────────┴────────────┴───────────────────┘
Execution time: 7.5713 seconds


In [17]:
print(type(pl_summary))

<class 'polars.dataframe.frame.DataFrame'>


In [13]:
start_time = time.time()
# Wrap Polars dataframes with Narwhals and reuse the same transformation pipeline
df_sales_nw_pl = nw.from_native(pl_sales)
df_regions_nw_pl = nw.from_native(pl_regions)
df_summary_pl = (df_sales_nw_pl.join(df_regions_nw_pl, on="id", how="inner")
                                .filter(nw.col("value") > 0)
                                .with_columns((nw.col("value") / pl_sales["value"].mean()).alias("value_norm")) 
                                .group_by("region")
                                .agg(
                                    nw.col("value").sum().alias("total_value"),
                                    nw.col("value").mean().alias("avg_value"),
                                    nw.len().alias("transaction_count")
                                ))
summary_pl_native = nw.to_native(df_summary_pl)  # this will be a Polars DataFrame
print("Summary by region (via Narwhals on Polars):\n", summary_pl_native)
end_time = time.time()
print(f"Execution time: {end_time - start_time:.4f} seconds")

Summary by region (via Narwhals on Polars):
 shape: (4, 4)
┌────────┬─────────────┬────────────┬───────────────────┐
│ region ┆ total_value ┆ avg_value  ┆ transaction_count │
│ ---    ┆ ---         ┆ ---        ┆ ---               │
│ str    ┆ f64         ┆ f64        ┆ u32               │
╞════════╪═════════════╪════════════╪═══════════════════╡
│ West   ┆ 5.1014e9    ┆ 102.75133  ┆ 49647685          │
│ East   ┆ 4.8789e9    ┆ 102.760417 ┆ 47478583          │
│ South  ┆ 5.0651e9    ┆ 102.75546  ┆ 49293018          │
│ North  ┆ 5.0386e9    ┆ 102.766448 ┆ 49029396          │
└────────┴─────────────┴────────────┴───────────────────┘
Execution time: 10.4196 seconds


In [18]:
print(type(df_summary_pl))

<class 'narwhals.dataframe.DataFrame'>


In [ ]:
# CSV Loading Benchmark (Notebook/VS Code friendly)
# pandas vs polars vs narwhals(pandas) vs narwhals(polars)

from __future__ import annotations
from pathlib import Path
import time, statistics as stats, json
from typing import Callable, List, Dict

import pandas as pd
import polars as pl
import narwhals as nw

# ---------- timing helpers ----------
def _time_once(fn: Callable[[], object]) -> float:
    t0 = time.perf_counter(); _ = fn(); return time.perf_counter() - t0

def time_fn(fn: Callable[[], object], repeats: int = 5, warmup: int = 1) -> float:
    for _ in range(max(0, warmup)): _ = _time_once(fn)      # warm-up (I/O cache, imports)
    return stats.median([_time_once(fn) for _ in range(max(1, repeats))])

# ---------- loaders (mirror your four cases) ----------
def pandas_loader(p: str):             return pd.read_csv(p)
def polars_loader(p: str):             return pl.read_csv(p)
def narwhals_pandas_loader(p: str):    return nw.from_native(pd.read_csv(p))
def narwhals_polars_loader(p: str):    return nw.from_native(pl.read_csv(p))

LOADERS: List[tuple[str, Callable[[str], object]]] = [
    ("Pandas Loading Time", pandas_loader),
    ("Polars Loading Time", polars_loader),
    ("Narwhals with Pandas Loading Time", narwhals_pandas_loader),
    ("Narwhals with Polars Loading Time", narwhals_polars_loader),
]

# ---------- benchmarking core ----------
def benchmark_csv_loading(
    files: list[str | Path] | None = None,
    *,
    repeats: int = 5,
    warmup: int = 1,
    save_csv: str | None = "loading_benchmark.csv",
    save_json: str | None = "loading_benchmark.json",
) -> pd.DataFrame:
    """
    Returns a DataFrame with columns:
    File | Pandas Loading Time | Polars Loading Time | Narwhals with Pandas Loading Time | Narwhals with Polars Loading Time

    - If `files` is None, looks for defaults next to the notebook:
      cards_data.csv, transactions_data.csv, users_data.csv.
      If none found, falls back to all *.csv in the current directory.
    - Filters strictly to .csv files (prevents kernel JSON from being picked).
    """
    cwd = Path.cwd()

    if files:
        candidates = [Path(f).expanduser().resolve() for f in files]
    else:
        # Defaults first
        defaults = [cwd / "cards_data.csv", cwd / "transactions_data.csv", cwd / "users_data.csv"]
        candidates = [p for p in defaults if p.exists()]
        # Fallback to all CSVs in CWD if defaults missing
        if not candidates:
            candidates = sorted(p for p in cwd.glob("*.csv") if p.is_file())

    # Keep only existing CSVs
    files_resolved = [p for p in candidates if p.suffix.lower() == ".csv" and p.exists()]
    if not files_resolved:
        raise FileNotFoundError(
            "No CSV files found. Pass explicit paths, or place CSVs in the notebook folder."
        )

    rows: List[Dict[str, object]] = []
    for path in files_resolved:
        row: Dict[str, object] = {"File": path.name}
        for col_name, loader in LOADERS:
            try:
                elapsed = time_fn(lambda p=str(path): loader(p), repeats=repeats, warmup=warmup)
                row[col_name] = round(elapsed, 6)
            except Exception as e:
                row[col_name] = None
                print(f"[ERROR] {col_name} failed for {path}: {e}")
        rows.append(row)

    df = pd.DataFrame(rows, columns=[
        "File",
        "Pandas Loading Time",
        "Polars Loading Time",
        "Narwhals with Pandas Loading Time",
        "Narwhals with Polars Loading Time",
    ])

    # Persist (optional)
    if save_csv:
        df.to_csv(save_csv, index=False)
    if save_json:
        with open(save_json, "w") as f:
            json.dump(rows, f, indent=2)

    return df


In [23]:
import os
CWD = os.getcwd()
card_file = os.path.join(CWD, "cards_data.csv")
transactions_file = os.path.join(CWD, "transactions_data.csv")
users_file = os.path.join(CWD, "users_data.csv")

In [24]:
benchmark_csv_loading(
        files=[
                card_file,
                transactions_file,
                users_file,
    ],
        repeats=5, warmup=1)

,File,Pandas Loading Time,Polars Loading Time,Narwhals with Pandas Loading Time,Narwhals with Polars Loading Time
0,cards_data.csv,0.006582,0.001743,0.006101,0.001580
1,transactions_data.csv,10.614732,0.569094,10.125354,0.507129
2,users_data.csv,0.004857,0.001856,0.004841,0.001775
